<a href="https://colab.research.google.com/github/vy-phung/genomic-data-science-project-about-fetus-and-adult/blob/main/Predicted_and_classified_genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction and Classification from fetal and adult genes 

## Objectives

I will use 3 types of genomic table data to identify if we can predict ages and classify age_group (fetus or adult) and genders of these 10 samples: 
1. Edata_tr.csv (15559 genes, haven't statistically analyzed but already filtered)
2. Data for regulated gene.csv (5566 genes, already statistically analyzed)
3. 2 most up-down regulated genes (2 genes, extracting from data for regulated gene.csv)

*Getting data in Github: genomic data (tidy_data folder); phenotype data (sample_data folder)*

**Steps:**
*   Classifying age_group (fetus or adult) of samples and gender (female or male) by using Logistic regression
*   Predicting age which is continuous data by using Multi linear regression, Ridge Regression with the support of Grid alpha search, and Random Forest Regressor


<h1>Table of content</h1>
<ul>
    <li><a href="#ref1">Part 1: Algorithm </a></li>
    <li><a href="#ref2">Part 2: Analyzing table type 1 </a></li>
    <li><a href="#ref3">Part 3: Analyzing table type 2 </a></li>
    <li><a href="#ref4">Part 4: Analyzing table type 3</a></li>



<h2>Part 1: Predicting and Classifying Algorithm</h2>

In [ ]:
#@title Import packages 
# Package
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
#@title Logistic regression & Multi linear regression
class Prediction:
  def __init__(self,x,y,lr):
    self.x = x
    self.y = y
    self.lr = lr
  def train_test(self):
      # lr is regressor used to predict or classify 
    x_train, x_test, y_train, y_test = train_test_split(self.x, self.y, test_size=0.20, random_state=0)
    self.lr.fit(x_train,y_train)
    self.x_test = x_test
    self.x_train = x_train
    self.y_test = y_test
    self.y_train = y_train
  def predict(self,x,y): 
    self.train_test() 
    if str(x.values) == str(self.x.values) and str(y.values) == str(self.y.values):
      x = self.x_test
      y = self.y_test
      print("number of test samples :", self.x_test.shape[0])
      print("number of training samples:",self.x_train.shape[0])
    print("actual data: ",y.values)
    print("predicted data: ",self.lr.predict(x)) 
  def cnf(self,x,y):
    if y.iloc[:,0].dtype == 'O': # logistic regression for binary variable
      self.train_test()
      if str(x.values) == str(self.x.values) and str(y.values) == str(self.y.values):
        self.x == x
        self.y == y 
      print("confusion matrix:",metrics.confusion_matrix(y, self.lr.predict(x)))
  def score(self,x,y):
    if y.iloc[:,0].dtype == 'float64': # regression for continuous data 
      self.train_test()
      if str(x.values) == str(self.x.values) and str(y.values) == str(self.y.values):
        print('The R-square of testing data: ', self.lr.score(self.x_test, self.y_test))
        print('The R-square of training data: ', self.lr.score(self.x_train, self.y_train))
        print('The mean square error of actual and predicted fitting data is: ', \
          mean_squared_error(self.y, self.lr.predict(self.x)))
        return
      print('The R-square: ', self.lr.score(x, y))

In [ ]:
#@title Ridge regression and Grid Search
class other_prediction:
  def __init__(self,x,y,parameters):
    self.x = x
    self.y = y
    self.parameters = parameters
  def train_test(self):
      # lr is regressor used to predict or classify 
    x_train, x_test, y_train, y_test = train_test_split(self.x, self.y, test_size=0.20, random_state=0)
    RR=Ridge()
    self.Grid = GridSearchCV(RR, self.parameters,cv=4)
    self.x_test = x_test
    self.x_train = x_train
    self.y_test = y_test
    self.y_train = y_train
  def Ridge(self,x,y):
    if y.iloc[:,0].dtype == 'float64':
      self.train_test()
      self.Grid.fit(self.x, self.y)
      BestRR=self.Grid.best_estimator_
      RigeModel=Ridge(alpha=BestRR.alpha)
      RigeModel.fit(self.x, self.y)
      print(BestRR)
      if str(x.values) == str(self.x.values) and str(y.values) == str(self.y.values):
        x = self.x_test
        y = self.y_test
      print("R^2 of data:",RigeModel.score(x, y))
      print("predict data:",RigeModel.predict(x))
      print("actual testing data:",y.values)

<h2>Part 2: Table type 1</h2>

#### <b>Data processing<b/>

In [ ]:
edata = pd.read_csv("edata_tr.csv")
edata

,Unnamed: 0,SRR1554534,SRR1554535,SRR1554568,SRR1554561,SRR1554567,SRR1554536,SRR1554541,SRR1554539,SRR1554538,SRR1554537
0,A1BG,9.143633,8.583227,8.105891,9.206773,6.995598,8.762864,8.373665,8.429514,7.632873,8.489938
1,ADA,8.646385,9.145974,7.303994,8.001912,7.351627,9.770115,7.843839,8.253862,7.831596,7.197164
2,CDH2,13.562468,13.585906,14.957846,13.652418,14.829826,13.812075,14.545283,13.717079,14.708753,15.051336
3,AKT3,13.479431,13.159273,15.876052,13.925121,14.376621,12.332018,15.204848,14.383520,14.749481,15.584859
4,ZBTB11-AS1,7.746129,7.539217,7.489181,7.323988,7.706548,7.731734,7.684944,7.654166,7.625887,7.705363
...,...,...,...,...,...,...,...,...,...,...,...
15554,CDH1,9.283095,9.269662,10.242205,8.787615,10.671178,9.219568,9.003333,8.933461,9.185240,9.167789
15555,SLC12A6,13.031252,12.899341,13.801854,13.197724,13.451059,12.386146,13.613920,13.345966,13.644767,13.585326
15556,PTBP3,11.367093,11.867973,13.315986,11.766260,12.716064,11.924352,12.990262,12.386899,12.840028,12.564455
15557,DGCR2,15.038492,14.455368,13.976339,15.153004,13.996213,14.484859,13.867110,14.553994,13.960779,14.168734


In [ ]:
edata = edata.set_index("Unnamed: 0")
edata.index.names  = [None]
edata_T = edata.T
edata_T

,A1BG,ADA,CDH2,AKT3,ZBTB11-AS1,MED6,NAALAD2,DDTL,NAALADL1,NINJ2-AS1,ACOT8,ABI1,GNPDA1,MIMT1,KCNE3,ZBTB33,SNHG8,GTF2IP4,CDH3,TANK,HAVCR1P1,POM121C,ZSCAN30,MCTS2P,SRA1,UCKL1-AS1,TMEM170B,SNORD58C,ZGLP1,FAM86JP,LOC100126784,C8orf88,PLCE1-AS1,FAM229A,JAZF1-AS1,LOC100128164,TMPO-AS1,LINC02731,ZNF667-AS1,LOC100128253,...,EXOG,XYLB,OXSR1,GFPT2,CRYZL1,WDR1,AMMECR1,CDC25C,GOLGA5,HS3ST4,HS3ST2,HS3ST1,USP15,CDC27,USP3,MVP,SLC23A2,SLC23A1,THRAP3,MED12,MED13,CDC34,NR1I3,NUP153,CCS,NR1D2,RBX1,CDC42,DOP1B,THOC1,REC8,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,PTBP3,DGCR2,CASP8AP2
SRR1554534,9.143633,8.646385,13.562468,13.479431,7.746129,10.965760,10.367572,7.328231,6.963692,8.822544,11.946038,13.366491,12.071443,8.185792,7.299651,11.724310,11.040949,9.332611,10.056517,11.043564,6.306034,12.131384,11.187060,8.428487,11.774578,5.942993,12.447507,5.990280,9.813163,8.156964,11.546933,5.310318,5.012922,8.198675,8.275069,7.411890,6.364689,9.262248,11.996517,8.508692,...,10.069137,8.260353,12.261088,11.033097,11.745050,13.960913,8.851549,4.834219,11.609201,13.196758,12.322287,10.436515,11.510020,12.598076,9.241031,11.818152,14.454579,6.029861,14.044886,12.553101,12.546297,13.108467,7.783518,12.263514,11.977166,13.057045,12.213122,13.563907,12.427323,11.218925,11.567748,10.500004,15.010639,13.123384,11.748144,9.283095,13.031252,11.367093,15.038492,11.208092
SRR1554535,8.583227,9.145974,13.585906,13.159273,7.539217,10.953407,10.644661,7.691085,6.885521,9.179721,11.757942,13.758680,12.430628,6.485860,7.174165,12.137270,11.156462,9.108857,8.777516,11.573996,5.659820,11.713654,11.583213,8.646386,11.696444,6.466384,13.535621,6.264076,9.802732,7.660717,11.087468,6.394250,5.049559,7.810233,8.786065,6.888673,6.585095,9.554467,12.107183,7.520252,...,10.350487,8.431188,12.252010,11.375670,11.812673,13.572111,9.177446,5.725231,12.156370,12.959437,12.410400,9.938927,12.066043,13.211498,9.896571,11.529537,14.225245,6.335155,13.753086,12.006727,12.878277,12.702216,7.319999,12.587600,11.735796,13.893709,12.673437,14.078996,12.243227,11.659515,11.144585,10.362881,15.207310,13.417473,12.330147,9.269662,12.899341,11.867973,14.455368,11.459779
SRR1554568,8.105891,7.303994,14.957846,15.876052,7.489181,11.062971,9.091478,7.186178,6.567708,8.099645,10.652184,12.833693,12.045205,8.525005,7.401378,13.056753,10.896357,9.652456,9.948553,12.237046,6.912586,12.078191,12.612542,9.491832,12.062338,7.420499,13.577083,6.726470,10.167270,6.409820,9.171032,7.553961,6.791164,7.500049,4.925746,6.790891,8.339422,7.691874,12.001934,5.392324,...,11.014434,9.493641,13.640403,10.264169,11.568180,13.661309,10.381393,9.036064,12.096830,13.808086,9.569563,10.199837,13.293106,13.498710,12.472372,8.653635,14.039276,7.900844,14.713353,13.746716,14.387287,12.293468,7.643377,13.459464,11.521068,12.491744,11.777826,14.129060,14.284260,12.559479,13.256709,11.223629,16.253288,14.440244,14.061450,10.242205,13.801854,13.315986,13.976339,12.916206
SRR1554561,9.206773,8.001912,13.652418,13.925121,7.323988,10.825176,10.406496,7.305990,6.689252,8.340297,12.014095,13.464797,12.325439,6.562989,7.236675,11.858819,10.746418,8.524623,9.883197,10.949873,6.030009,12.425645,11.040663,8.539710,11.678305,7.618927,12.740738,5.806708,10.260433,8.057940,11.414649,5.844022,4.581718,8.199565,8.083382,7.354909,6.401091,9.734726,11.736873,8.910149,...,10.316555,9.095420,12.113526,11.062619,11.844331,13.799661,8.785126,5.243429,11.545216,12.645491,12.061151,10.549871,11.786105,12.582161,8.911123,11.170068,14.513305,6.162469,13.985093,12.670834,12.884486,13.156844,7.894102,12.546458,12.006163,13.571330,12.112764,13.609118,12.720381,10.998715,11.434048,10.645623,14.825710,13.043291,12.223982,8.787615,13.197724,11.766260,15.153004,11.188994
SRR1554567,6.995598,7.351627,14.829826,14.376621,7.706548,11.161363,10.659595,7.545062,7.167443,8.226544,10.470291,13.058067,11.799100,8.035972,7.522792,12.975194,10.914658,9.626314,10.214180,11.847383,7.252727,11.747622,12.383769,9.571803,11.834974,7.270623,13.558488,6.698271,10.177543,6.812443

In [ ]:
pheno = pd.read_csv("/content/pheno_sample.csv")
pheno = pheno.set_index("Unnamed: 0")
pheno.index.names = [None]
pheno

,fetus_adult,age_group,age,sex,gender
SRR1554534,adult,1,40.4200,male,1
SRR1554535,adult,1,41.5800,male,1
SRR1554568,fetus,0,-0.4986,male,1
SRR1554561,adult,1,43.8800,male,1
SRR1554567,fetus,0,-0.4027,male,1
SRR1554536,adult,1,44.1700,female,0
SRR1554541,fetus,0,-0.3836,male,1
SRR1554539,adult,1,36.5000,female,0
SRR1554538,fetus,0,-0.4027,female,0
SRR1554537,fetus,0,-0.3836,female,0


In [ ]:
x = edata_T

#### <b>Logistic regression<b/>

**Classifying fetus or adult based on age_group**

In [ ]:
y = pheno[["age_group"]]
y

,age_group
SRR1554534,1
SRR1554535,1
SRR1554568,0
SRR1554561,1
SRR1554567,0
SRR1554536,1
SRR1554541,0
SRR1554539,1
SRR1554538,0
SRR1554537,0


In [ ]:
lr = LogisticRegression()
p = Prediction(x,y,lr)
p.predict(x,y)
p.cnf(x,y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


number of test samples : 2
number of training samples: 8
actual data:  [[0]
 [0]]
predicted data:  [0 0]


**Classifying gender of samples**

In [ ]:
y_gender = pheno[["gender"]]
y_gender

,gender
SRR1554534,1
SRR1554535,1
SRR1554568,1
SRR1554561,1
SRR1554567,1
SRR1554536,0
SRR1554541,1
SRR1554539,0
SRR1554538,0
SRR1554537,0


In [ ]:
p = Prediction(x,y_gender,lr)
p.predict(x,y_gender)
p.cnf(x,y_gender)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


number of test samples : 2
number of training samples: 8
actual data:  [[1]
 [0]]
predicted data:  [0 0]


#### <b>Predicting age (continuous data)<b/>

In [ ]:
y_age = pheno[["age"]]
y_age

,age
SRR1554534,40.4200
SRR1554535,41.5800
SRR1554568,-0.4986
SRR1554561,43.8800
SRR1554567,-0.4027
SRR1554536,44.1700
SRR1554541,-0.3836
SRR1554539,36.5000
SRR1554538,-0.4027
SRR1554537,-0.3836


**Multi linear regression**

In [ ]:
lr = LinearRegression()
p = Prediction(x,y_age,lr)
p.predict(x,y_age)
p.score(x,y_age)

number of test samples : 2
number of training samples: 8
actual data:  [[-0.4986]
 [-0.4027]]
predicted data:  [[-3.38719242]
 [-1.75369115]]
The R-square of testing data:  -2210.4501135869386
The R-square of training data:  1.0
The mean square error of actual and predicted fitting data is:  1.0169143259568714


The R-square of testing data:  -2210.4501135869386

The R-square of training data:  1.0

We can see that R^2 of testing data is negatively large, while that of training data is fit to statistical model

=> It is overfitting

**Using grid search to find suitable alpha for ridge model**

In [ ]:
parameters= [{'alpha': [0.001,0.1,1, 10, 100, 1000, 10000, 100000, 100000]}]
o = other_prediction(x,y_age,parameters)
o.Ridge(x,y_age)

Ridge(alpha=1000)
R^2 of data: -31.726080041146417
predict data: [[-0.88566156]
 [-0.428606  ]]
actual testing data: [[-0.4986]
 [-0.4027]]


R^2 of data: -31.726080041146417

We can see that using Ridge model combined with alpha 1000 can increase R^2
 
=> It might be better to use than multi linear regression model.


<h2>Part 3: Table type 2</h2>

#### <b>Data processing<b/>

In [ ]:
up_down = pd.read_csv("data for regulated gene.csv")
up_down

,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,SRR1554534,SRR1554535,SRR1554568,SRR1554561,SRR1554567,SRR1554536,SRR1554541,SRR1554539,SRR1554538,SRR1554537
0,A1BG,380.846902,-1.102936,0.407692,-2.705317,6.823930e-03,1.190664e-02,9.143633,8.583227,8.105891,9.206773,6.995598,8.762864,8.373665,8.429514,7.632873,8.489938
1,A2M,13185.383185,-1.670680,0.420489,-3.973179,7.091974e-05,1.717291e-04,13.657416,13.795969,12.584194,13.003245,12.597035,15.235020,12.938130,13.881265,12.930901,12.535831
2,A2ML1,484.632776,-2.748295,0.569283,-4.827644,1.381576e-06,4.322248e-06,8.848175,9.062477,6.909735,8.644313,7.457088,10.622862,6.885452,7.708383,7.500564,7.351208
3,A4GALT,412.311453,-2.790862,0.571964,-4.879432,1.063917e-06,3.379052e-06,8.868851,8.344972,6.895125,8.568670,6.710507,10.362960,7.260562,7.533763,7.292369,6.723675
4,AARD,124.885643,-2.105662,0.537200,-3.919699,8.865947e-05,2.115377e-04,7.347104,7.314591,5.402734,6.636227,6.395016,7.306073,6.484457,7.470666,5.264764,5.737391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7026,ZXDA,1338.880407,1.446283,0.201328,7.183722,6.783895e-13,4.459430e-12,9.298298,9.408328,10.695708,9.704292,10.691396,9.581422,10.874795,9.996956,10.778896,10.978820
7027,ZXDB,2840.638643,1.029280,0.251155,4.098180,4.164116e-05,1.045492e-04,10.528675,11.216468,11.697161,10.929075,11.812847,10.392533,11.949005,11.379329,11.716040,11.857990
7028,ZXDC,14083.362352,1.596761,0.129687,12.312466,7.761457e-35,3.260866e-33,12.904692,12.734604,14.242381,12.935801,14.026589,12.953526,14.199765,12.951150,14.424315,14.336139
7029,ZYX,14229.672689,-1.002155,0.256375,-3.908937,9.270299e-05,2.205726e-04,14.633231,13.965675,13.345845,14.427006,13.168138,13.685386,13.205510,13.632358,13.208680,13.456611


In [ ]:
select = ["Unnamed: 0","SRR1554534",	"SRR1554535",	"SRR1554568",	"SRR1554561",	"SRR1554567",	"SRR1554536",	"SRR1554541",	"SRR1554539",	"SRR1554538",	"SRR1554537"]
up_down1 = up_down[select]
up_down1 = up_down1.set_index("Unnamed: 0")
up_down1.index.names = [None]
up_down_T = up_down1.T
up_down_T

,A1BG,A2M,A2ML1,A4GALT,AARD,AARS1,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA2,ABCA6,ABCA7,ABCA8,ABCA9,ABCB1,ABCB7,ABCC12,ABCC4,ABCC8,ABCC9,ABCG2,ABCG4,ABHD11,ABHD12,ABHD12B,ABHD14A,ABHD14B,ABHD15,ABHD16B,ABHD17A,ABHD18,ABHD8,ABI3,ABI3BP,ABL2,ABLIM2,ABR,ABRACL,...,ZNF878,ZNF879,ZNF880,ZNF883,ZNF888,ZNF890P,ZNF90,ZNF91,ZNF92,ZNF93,ZNHIT1,ZNHIT2,ZNRF1,ZNRF2,ZNRF3,ZRANB2-AS1,ZSCAN12,ZSCAN16,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN9,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM8-AS1,ZUP1,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYX,ZZZ3
SRR1554534,9.143633,13.657416,8.848175,8.868851,7.347104,15.757594,15.412314,15.773004,10.728758,8.459024,6.127944,16.924871,6.642122,11.876442,11.226060,10.146097,11.725727,10.232176,9.959768,8.642513,13.262715,11.017764,12.156003,12.361851,10.660498,14.027339,8.701414,11.986268,10.572723,9.056314,9.861490,12.058016,9.233208,14.098318,7.786744,7.519580,12.477906,14.127248,16.562266,10.143301,...,5.523122,8.388920,9.117129,9.318429,7.031279,7.694471,5.447482,10.384837,9.385131,7.455699,11.989397,9.976802,13.007817,9.953358,12.479115,5.873261,9.956498,8.342552,10.050968,8.445623,9.713292,9.671006,7.658145,11.056144,11.531460,11.187060,9.657207,9.298647,11.358897,10.997099,12.237565,6.544398,9.872600,9.203633,9.158430,9.298298,10.528675,12.904692,14.633231,11.142157
SRR1554535,8.583227,13.795969,9.062477,8.344972,7.314591,15.345289,15.065392,15.345071,11.097286,8.806715,6.304167,16.764446,7.367755,11.504536,12.397963,10.500113,11.504491,10.476153,9.713523,9.107198,12.773792,11.002192,12.353510,11.955876,10.431284,13.922281,8.933736,11.841070,10.369419,9.143898,9.390017,11.550377,9.317387,13.556946,7.055041,8.419397,12.318609,13.426870,16.039723,10.826507,...,6.584933,9.262461,9.203113,9.805414,7.386525,6.746905,6.036277,11.065840,10.845381,7.943450,11.735374,9.699339,12.606994,10.172354,12.324561,7.158877,10.243413,8.677554,9.795162,8.263585,9.464862,9.680491,7.331028,11.578808,11.704103,11.583213,9.615276,9.454976,10.480033,10.825541,12.483370,6.814309,10.563563,10.563658,9.266755,9.408328,11.216468,12.734604,13.965675,11.893657
SRR1554568,8.105891,12.584194,6.909735,6.895125,5.402734,14.133470,11.532492,14.755580,13.191035,6.858484,7.621509,14.372198,6.097790,11.133889,8.692355,9.506912,9.500953,11.538425,5.277975,10.846871,10.464994,9.921029,10.012782,10.405635,9.161784,11.737641,3.264354,10.270335,9.561828,7.882171,8.783332,9.927939,10.521080,11.857025,6.375065,9.691310,13.387431,10.465655,15.238726,12.014159,...,7.839871,10.127181,11.089222,11.054856,8.429374,5.974485,8.839597,13.197695,12.082913,10.808293,10.895053,8.697400,14.001511,11.231796,11.589324,8.347241,12.090830,10.170591,12.288275,9.769823,10.585639,10.576931,10.375166,12.529951,12.852354,12.612542,11.566507,10.154758,12.185891,13.014882,13.548023,7.629274,11.259243,11.688186,11.768482,10.695708,11.697161,14.242381,13.345845,12.719315
SRR1554561,9.206773,13.003245,8.644313,8.568670,6.636227,15.908997,15.465764,15.899338,10.717510,8.575097,5.908803,16.784595,6.879113,12.057728,11.010436,10.494653,11.084899,9.739800,10.092062,8.077065,13.661388,10.231561,11.516742,12.396533,10.475263,13.848728,8.642972,11.968709,10.327511,8.692463,9.800060,12.277649,9.003095,14.280003,6.652067,7.843736,12.702621,14.271265,16.907170,10.242545,...,5.708142,8.073806,9.156198,9.659393,6.808310,7.877490,5.726660,10.788137,9.020034,7.214739,11.777721,10.451524,12.989843,10.252754,12.057338,6.447057,9.869246,7.923305,9.945018,9.097662,9.199770,9.875337,8.230564,11.173858,11.610486,11.040663,9.007093,9.615308,11.489777,10.879565,12.278762,6.672439,9.894020,9.646344,8.770140,9.704292,10.929075,12.935801,14.427006,11.499921
SRR1554567,6.995598,12.597035,7.457088,6.710507,6.395016,14.037013,11.828298,14.695775,12.993483,7.561909,7.299447,14.174753,6.097308,9.223357,10.005713,9.447863,9.724329,11.047798,7.269236,10.216889,11.079017,10.008156,10.814944,10.334022,9.238554,12.053859,4.516572,10.446782,9.577568,7.492575,8.560335,9.900396,10.267065,11.787334,6.165152,10.0415

In [ ]:
x1 = up_down_T

#### <b>Logistic regression<b/>

**Classifying fetus or adult based on age_group**

In [ ]:
lr = LogisticRegression()
p = Prediction(x1,y,lr)
p.predict(x1,y)
p.cnf(x1,y)

number of test samples : 2
number of training samples: 8
actual data:  [[0]
 [0]]
predicted data:  [0 0]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Classifying gender of samples**

In [ ]:
p = Prediction(x1,y_gender,lr)
p.predict(x1,y_gender)
p.cnf(x1,y_gender)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


number of test samples : 2
number of training samples: 8
actual data:  [[1]
 [0]]
predicted data:  [0 1]


#### <b>Predicting age (continuous data)<b/>

**Multi linear regression**

In [ ]:
lr = LinearRegression()
p = Prediction(x1,y_age,lr)
p.predict(x1,y_age)
p.score(x1,y_age)

number of test samples : 2
number of training samples: 8
actual data:  [[-0.4986]
 [-0.4027]]
predicted data:  [[-3.72575422]
 [-1.9660493 ]]
The R-square of testing data:  -2795.314245992872
The R-square of training data:  1.0
The mean square error of actual and predicted fitting data is:  1.2858585410344976


The R-square of testing data:  -2759.9297929965064

The R-square of training data:  1.0

We can see that R^2 of testing data is negatively large, while that of training data is exactly fit which also means statistical model fits

It is overfitting

**Using grid search to find suitable alpha for ridge model to predict age of sample**

In [ ]:
parameters= [{'alpha': [0.001,0.1,1, 10, 100, 1000, 10000, 100000, 100000]}]
o = other_prediction(x1,y_age,parameters)
o.Ridge(x1,y_age)

Ridge(alpha=1000)
R^2 of data: -94.94043343888032
predict data: [[-1.15500159]
 [-0.50423776]]
actual testing data: [[-0.4986]
 [-0.4027]]


R^2 of data: -94.94043343888032

We can see that using Ridge model combined with alpha 1000 can increase the R^2, and reduce overfitting compared with multi linear regression 

<h2>Part 4: Table type 3</h2>

#### <b>Data processing <b/>

In [ ]:
# The most up-regulated genes
up = up_down[up_down['log2FoldChange'] > 1]
up = up[up["padj"] == min(up["padj"])]
up = up.set_index("Unnamed: 0")
up.index.names = [None]
up

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,SRR1554534,SRR1554535,SRR1554568,SRR1554561,SRR1554567,SRR1554536,SRR1554541,SRR1554539,SRR1554538,SRR1554537
ST8SIA2,21470.98555,7.44298,0.2003,37.159148,3.119957e-302,4.823766e-298,8.614598,8.485263,14.645985,8.707785,14.756824,8.39641,15.159507,8.971893,14.917903,15.015816


In [ ]:
# The most down-regulated genes
down = up_down[up_down['log2FoldChange'] < -1]
down = down[down["padj"] == min(down["padj"])]
down = down.set_index("Unnamed: 0")
down.index.names = [None]
down

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,SRR1554534,SRR1554535,SRR1554568,SRR1554561,SRR1554567,SRR1554536,SRR1554541,SRR1554539,SRR1554538,SRR1554537
BCL2L2,21968.220153,-2.676407,0.088315,-30.305208,9.788276e-202,3.783413e-198,14.967316,14.943844,12.776112,15.151775,12.71442,14.999478,12.761278,15.053436,12.718466,12.793366


In [ ]:
up1 = up[["SRR1554534",	"SRR1554535",	"SRR1554568",	"SRR1554561",	"SRR1554567",	"SRR1554536",	"SRR1554541",	"SRR1554539",	"SRR1554538",	"SRR1554537"]]
up1_T = up1.T
up1_T

,ST8SIA2
SRR1554534,8.614598
SRR1554535,8.485263
SRR1554568,14.645985
SRR1554561,8.707785
SRR1554567,14.756824
SRR1554536,8.396410
SRR1554541,15.159507
SRR1554539,8.971893
SRR1554538,14.917903
SRR1554537,15.015816


In [ ]:
down1 = down[["SRR1554534",	"SRR1554535",	"SRR1554568",	"SRR1554561",	"SRR1554567",	"SRR1554536",	"SRR1554541",	"SRR1554539",	"SRR1554538",	"SRR1554537"]]
down1_T = down1.T
down1_T

,BCL2L2
SRR1554534,14.967316
SRR1554535,14.943844
SRR1554568,12.776112
SRR1554561,15.151775
SRR1554567,12.714420
SRR1554536,14.999478
SRR1554541,12.761278
SRR1554539,15.053436
SRR1554538,12.718466
SRR1554537,12.793366


In [ ]:
x2 = pd.concat([up1_T,down1_T],axis=1)
x2

,ST8SIA2,BCL2L2
SRR1554534,8.614598,14.967316
SRR1554535,8.485263,14.943844
SRR1554568,14.645985,12.776112
SRR1554561,8.707785,15.151775
SRR1554567,14.756824,12.714420
SRR1554536,8.396410,14.999478
SRR1554541,15.159507,12.761278
SRR1554539,8.971893,15.053436
SRR1554538,14.917903,12.718466
SRR1554537,15.015816,12.793366


#### <b>Logistic regression<b/>

**Classifying by age_group**

In [ ]:
lr = LogisticRegression()
p = Prediction(x2,y,lr)
p.predict(x2,y)
p.cnf(x2,y)

number of test samples : 2
number of training samples: 8
actual data:  [[0]
 [0]]
predicted data:  [0 0]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Classifying by gender**

In [ ]:
# gender
p = Prediction(x2,y_gender,lr)
p.predict(x2,y_gender)
p.cnf(x2,y_gender)

number of test samples : 2
number of training samples: 8
actual data:  [[1]
 [0]]
predicted data:  [1 1]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### <b>Predicting age (continuous data)<b/>

**Multi linear regression**

In [ ]:
lr = LinearRegression()
p = Prediction(x2,y_age,lr)
p.predict(x2,y_age)
p.score(x2,y_age)

number of test samples : 2
number of training samples: 8
actual data:  [[-0.4986]
 [-0.4027]]
predicted data:  [[ 1.38679745]
 [-0.25893323]]
The R-square of testing data:  -776.5288225177012
The R-square of training data:  0.9934719800426676
The mean square error of actual and predicted fitting data is:  2.511263182559502


The R-square of testing data:  -776.5288225177012

We can see that R^2 of testing data is negatively large, while that of training data nearly fits statistical model

=> It is overfitting

**Using grid search to find suitable alpha for ridge model**

In [ ]:
parameters= [{'alpha': [0.001,0.1,1, 10, 100, 1000, 10000, 100000, 100000]}]
o = other_prediction(x2,y_age,parameters)
o.Ridge(x2,y_age)

Ridge(alpha=1)
R^2 of data: -684.0498220950029
predict data: [[ 1.27614849]
 [-0.42280739]]
actual testing data: [[-0.4986]
 [-0.4027]]


R^2 of data: -684.0498220950029

**Random Forest Regression**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
lr = RandomForestRegressor(n_estimators=1)
p = Prediction(x2,y_age,lr)
p.predict(x2,y_age)
p.score(x2,y_age)

number of test samples : 2
number of training samples: 8
actual data:  [[-0.4986]
 [-0.4027]]
predicted data:  [-0.3836 -0.3836]
The R-square of testing data:  -1.8759972207754645
The R-square of training data:  0.9830837747102115
The mean square error of actual and predicted fitting data is:  5.582322500000003


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


Using random forest regression with estimators = 1, we have R-square of testing data:  -1.8759972207754645 

In this table type 3, the random forest regression is better than multi linear regression and ridge model. This is because its R^2 of testing data is the highest, but it's still negative.  